<a href="https://colab.research.google.com/github/bhavya-1204/AI-human-texts/blob/main/wikipedia_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36'}

In [3]:
def wikipedia_scrapping(names):
  data = []
  # input_link = input('paste link : ')
  for name in names:
    webpage = requests.get(f'https://en.wikipedia.org/wiki/{name}', headers=headers).text
    soup = BeautifulSoup(webpage, 'lxml')
    div = soup.find_all('div', class_='mw-content-ltr mw-parser-output')
    for i in div:
      p = i.find_all('p')
      for j in range(0, len(p)):
        data.append(p[j].text.strip())
  # for i in range(0, len(soup.find_all('p'))):
  #   data.append(soup.find_all('p')[i].text)
  data = pd.DataFrame(data)
  return data

In [4]:
names = [
  "Theory of Forms",
  "Phenomenology (philosophy)",
  "Existentialism",
  "Epistemology",
  "Metaphysics",
  "Ethics",
  "Philosophy of mind",
  "Philosophy of language",
  "Problem of induction",
  "Free will",
  "Mind–body problem",
  "Moral realism",
  "Nihilism",
  "Absurdism",
  "Critique of Pure Reason",
  "Being and Nothingness",
  "Tractatus Logico-Philosophicus",
  "Meditations on First Philosophy",
  "Phenomenology of Spirit",
  "Logical positivism",
  "Empiricism",
  "Rationalism",
  "Determinism",
  "Compatibilism",
  "Philosophical skepticism",
  "Gettier problem",
  "Ontology",
  "Teleology",
  "Philosophy of science",
  "Psychoanalysis",
  "Cognitive dissonance",
  "Behaviorism",
  "Human consciousness",
  "Theory of mind",
  "Intelligence",
  "Emotion",
  "Personality psychology",
  "Nature versus nurture",
  "Developmental psychology",
  "Self-consciousness",
  "Personal identity",
  "Psychological egoism",
  "Altruism",
  "Motivation",
  "Creativity",
  "Memory",
  "Perception",
  "Learning",
  "Human behavior",
  "Social cognition",
  "Moral psychology",
  "Social constructionism",
  "Symbolic interactionism",
  "Structural functionalism",
  "Conflict theory",
  "Cultural relativism",
  "Collective consciousness",
  "Social stratification",
  "Bureaucracy",
  "Power (social and political)",
  "Ideology",
  "Nation-state",
  "Modernity",
  "Globalization",
  "The Leviathan (Hobbes)",
  "Social contract",
  "Liberalism",
  "Conservatism",
  "Anarchism",
  "Totalitarianism",
  "Democracy",
  "Separation of powers",
  "Rule of law",
  "Sovereignty",
  "Civil disobedience",
  "Political legitimacy",
  "French Revolution",
  "Industrial Revolution",
  "Cold War",
  "Renaissance",
  "Age of Enlightenment",
  "Fall of the Roman Empire",
  "History of Western philosophy",
  "History of democracy",
  "History of science",
  "History of capitalism",
  "History of slavery",
  "History of colonialism",
  "History of nationalism",
  "History of human rights",
  "Historiography",
  "Oral history",
  "Capitalism",
  "Socialism",
  "Marxism",
  "Feminism",
  "Postmodernism",
  "Theory of relativity",
  "Evolution",
  "Big Bang",
  "Quantum mechanics",
  "Scientific method",
  "Falsifiability",
  "Chaos theory",
  "Game theory",
  "Systems theory",
  "Linguistics",
  "Syntax",
  "Semantics",
  "Pragmatics",
  "Language acquisition",
  "Universal grammar",
  "Structuralism",
  "Discourse analysis",
  "Semiotics",
  "Sign (semiotics)",
  "Meaning (linguistics)",
  "Discourse",
  "Narrative",
  "Rhetoric",
  "Hermeneutics",
  "Textual criticism",
  "Pragmatic meaning",
  "Speech act theory",
  "Grice's maxims",
  "Aesthetics",
  "Art criticism",
  "Literary theory",
  "Narratology",
  "Romanticism",
  "Realism (arts)",
  "Modernism",
  "Postmodern literature",
  "Tragedy",
  "Comedy",
  "Myth",
  "Allegory",
  "Natural law",
  "Legal positivism",
  "Justice",
  "Equity (law)",
  "Human rights",
  "Just war theory",
  "Bioethics",
  "Moral responsibility",
  "Punishment",
  "Social justice",
  'Charlie Kirk',
  'Donald Trump',
  'Elon Musk',
  'Taylor Swift',
  'Cristiano Ronaldo',
  'Lionel Messi',
  'MrBeast',
  'Joe Biden',
  'Kamala Harris',
  'Narendra Modi',
  'Vladimir Putin',
  'Pope Francis',
  'Virat Kohli',
  'Jeff Bezos',
  'Mark Zuckerberg',
  'Bill Gates',
  'Robert F. Kennedy Jr.',
  'Dwayne Johnson',
  'Selena Gomez',
  'Kylian Mbappé'
]

In [5]:
data = wikipedia_scrapping(names)

In [6]:
len(data)

13054

In [7]:
data.to_csv('Dset.csv', index=False)